In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [26]:
train_essay = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")

In [27]:
train_prompt = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")

In [28]:
train_essay

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [29]:
train_prompt

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [30]:
train_data = pd.merge(train_essay, train_prompt, on='prompt_id', how='inner')

In [31]:
train_data.head(2)

,id,prompt_id,text,generated,prompt_name,instructions,source_text
0,0059830c,0,Cars. Cars have been around since they became ...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,005db917,0,Transportation is a large necessity in most co...,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."


In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_val, y_train, y_val = train_test_split(train_data['text'], train_data['generated'], test_size=0.2, random_state=42)


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline

In [35]:
model = make_pipeline(TfidfVectorizer(), XGBClassifier())

In [36]:
model.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [37]:
val_predictions = model.predict_proba(X_val)[:, 1]


In [38]:
from sklearn.metrics import roc_auc_score


In [39]:
roc_auc = roc_auc_score(y_val, val_predictions)
print(f'Validation ROC AUC: {roc_auc}')

Validation ROC AUC: 0.23996350364963503


In [40]:
test_essays = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [41]:
test_predictions = model.predict_proba(test_essays['text'])[:, 1]

In [42]:
sample_submission = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')

In [43]:
submission_df = pd.DataFrame({'id': sample_submission['id'], 'generated': test_predictions})

In [44]:
submission_df.to_csv('submission.csv', index=False)

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [50]:
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [51]:
vectorizer = TfidfVectorizer()


In [52]:
X_train = vectorizer.fit_transform(train_data['text'])
X_val = vectorizer.transform(val_data['text'])
y_train = train_data['generated']
y_val = val_data['generated']

In [53]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [54]:
val_predictions_rf = rf_classifier.predict_proba(X_val)[:, 1]

In [55]:
roc_auc_rf = roc_auc_score(y_val, val_predictions_rf)
print(f'Random Forest Validation ROC AUC: {roc_auc_rf}')

Random Forest Validation ROC AUC: 0.9762773722627737


In [56]:
test_df = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [57]:
test_df

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [58]:
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')

In [59]:
sub

,id,generated
0,0000aaaa,0.1
1,1111bbbb,0.9
2,2222cccc,0.4


In [69]:
X_test_df = vectorizer.transform(test_df['text'])

In [70]:
test_predictions = rf_classifier.predict_proba(X_test_df)[:, 1]

In [71]:
submission_df = pd.DataFrame({'id': test_df['id'], 'generated': test_predictions})

In [72]:
print(submission_df)

         id  generated
0  0000aaaa        0.0
1  1111bbbb        0.0
2  2222cccc        0.0


In [73]:
submission_df.to_csv('submission_rf.csv', index=False)

In [75]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer

In [76]:
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['text'])

In [77]:
X_train = tokenizer.texts_to_sequences(train_data['text'])
X_val = tokenizer.texts_to_sequences(val_data['text'])

In [79]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [80]:
max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')

In [81]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data['generated'])
y_val = label_encoder.transform(val_data['generated'])

In [83]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [84]:
model= Sequential()
model.add(Embedding(input_dim=max_words, output_dim=50, input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [85]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [86]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))


Epoch 1/5
35/35 [==============================] - 17s 241ms/step - loss: 0.2052 - accuracy: 0.9837 - val_loss: 0.0570 - val_accuracy: 0.9928
Epoch 2/5
35/35 [==============================] - 5s 147ms/step - loss: 0.0077 - accuracy: 0.9991 - val_loss: 0.0554 - val_accuracy: 0.9928
Epoch 3/5
35/35 [==============================] - 4s 113ms/step - loss: 0.0077 - accuracy: 0.9991 - val_loss: 0.0532 - val_accuracy: 0.9928
Epoch 4/5
35/35 [==============================] - 2s 69ms/step - loss: 0.0075 - accuracy: 0.9991 - val_loss: 0.0513 - val_accuracy: 0.9928
Epoch 5/5
35/35 [==============================] - 3s 69ms/step - loss: 0.0075 - accuracy: 0.9991 - val_loss: 0.0515 - val_accuracy: 0.9928


In [87]:
val_predictions = model.predict(X_val)
roc_auc = roc_auc_score(y_val, val_predictions)
print(f'Validation ROC AUC: {roc_auc}')

9/9 [==============================] - 0s 4ms/step
Validation ROC AUC: 0.5948905109489051


In [89]:
X_test_ = tokenizer.texts_to_sequences(test_df['text'])
X_test_ = pad_sequences(X_test_, maxlen=max_length, padding='post')

In [90]:
test_predictions = model.predict(X_test_)

1/1 [==============================] - 0s 21ms/step


In [91]:
submission_df = pd.DataFrame({'id': test_df['id'], 'generated': test_predictions.flatten()})


In [92]:
print(submission_df)


         id  generated
0  0000aaaa   0.000947
1  1111bbbb   0.000947
2  2222cccc   0.000947


In [93]:
submission_df.to_csv('submission_keras_nlp.csv', index=False)

In [94]:
embedding_dim = 50
model_LLM = Sequential()
model_LLM.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_length))
model_LLM.add(LSTM(128, return_sequences=True))
model_LLM.add(LSTM(64))
model_LLM.add(Dense(32, activation='relu'))
model_LLM.add(Dropout(0.5))
model_LLM.add(Dense(1, activation='sigmoid'))

In [96]:
from tensorflow.keras.optimizers import Adam

In [97]:
model_LLM.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [98]:
model_LLM.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
35/35 [==============================] - 11s 198ms/step - loss: 0.1755 - accuracy: 0.9819 - val_loss: 0.0641 - val_accuracy: 0.9928
Epoch 2/10
35/35 [==============================] - 5s 140ms/step - loss: 0.0096 - accuracy: 0.9991 - val_loss: 0.0772 - val_accuracy: 0.9928
Epoch 3/10
35/35 [==============================] - 4s 124ms/step - loss: 0.0162 - accuracy: 0.9991 - val_loss: 0.0771 - val_accuracy: 0.9928
Epoch 4/10
35/35 [==============================] - 2s 66ms/step - loss: 0.0129 - accuracy: 0.9991 - val_loss: 0.0790 - val_accuracy: 0.9928
Epoch 5/10
35/35 [==============================] - 2s 66ms/step - loss: 0.0117 - accuracy: 0.9991 - val_loss: 0.0799 - val_accuracy: 0.9928
Epoch 6/10
35/35 [==============================] - 3s 97ms/step - loss: 0.0085 - accuracy: 0.9991 - val_loss: 0.0849 - val_accuracy: 0.9928
Epoch 7/10
35/35 [==============================] - 2s 61ms/step - loss: 0.0167 - accuracy: 0.9991 - val_loss: 0.0898 - val_accuracy: 0.9928
Epoch 8/1

In [99]:
val_predictions_LLM = model_LLM.predict(X_val)

9/9 [==============================] - 1s 7ms/step


In [100]:
X_test = tokenizer.texts_to_sequences(test_df['text'])
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

In [101]:
test_predictions_LLM = model_LLM.predict(X_test)


1/1 [==============================] - 0s 22ms/step


In [104]:
Submission__LLM = pd.DataFrame({'id': test_df['id'], 'generated': test_predictions_LLM.flatten()})


In [105]:
print(Submission__LLM)


         id  generated
0  0000aaaa   0.000095
1  1111bbbb   0.000095
2  2222cccc   0.000095


In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer